In [1]:
%load_ext autoreload
%autoreload 2

# Property Scraper

Author : Emanuele Cannizzaro

# Introduction

This notebook supports a set of tasks required to create the property databases stored at [Italia Estate Database](http://italia.estate-database.com/) and [real estate database](https://docs.google.com/spreadsheets/d/1BRloTbcVOFAL9up2wIsvaAjFuJep9f3TWQwp_f02ntw/edit#gid=1748870476).

The project also contains a [Google Lookerstudio dashboard](https://lookerstudio.google.com/u/0/reporting/2da6484e-c407-40aa-88e0-3ca0f6ddd31d/page/p_0pjh8udt4c).

It enriches the data from [Portale delle Vendite Pubbliche](https://pvp.giustizia.it/pvp/it/homepage.page).

![PVP](../images/pvp.png)


In [158]:
from bs4 import BeautifulSoup
# Reactor restart
#from crochet import setup, wait_for
from datetime import date, datetime
from glob import glob
from gspread import service_account
import json
#import linkcheck
from multiprocessing import Pool
import os
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess
#from scrapy.crawler import CrawlerRunner
#from scrapy.utils.project import get_project_settings
import subprocess
from subprocess import run
from tqdm.auto import tqdm

from property_scraper import PVP_URL_ROOTNAME, ROOT_FOLDER
from property_scraper import fix_hyperlinks, get_basename, get_filename_from_identifier, get_parameters
#from property_scraper.astegiudiziarie.spiders.search import AsteGiudiziarieSearchSpider
from property_scraper.process import Checker, Configurator, Extractor, Fixer
from property_scraper.pvp_scraper import PVPScraper
from property_scraper.store import SearchStore, SearchPropertyStore, PropertyStore

In [4]:
#setup()

In [5]:
pd.set_option('display.max_rows', None)

# Description

The following usecases are considered:

1. Prepare Scrapy Search Configuration;
1. Create or Update the search, search_property and property tables of the database;
1. Repair the downloaded HTML pages for the localhost;
1. Check the localhost server links;
1. Check the cloud web server links;
1. ...


## To do

1. To perform a new incremental search there is a need to understand when to stop downloading the search pages. 

1. Look for missing values 
if I am working on a Pandas dataframe, how can I identify whether there are missing rows in a column of continuous integer values? An elegant solution would use min, max and range(min, max+1).

A. ChatGPT
You can identify missing rows in a column of continuous integer values in a Pandas DataFrame by comparing the expected range of values to the actual values in the column. Here's an example:

In this example, we first create an example DataFrame with a column id containing continuous integer values. We then get the expected range of values using the range() function and convert it to a set. Next, we get the actual values in the id column of the DataFrame and convert it to a set. We then subtract the actual set from the expected set to get a set of missing values.

Finally, we check whether the missing set is non-empty and print out the missing values if any, or a message indicating that there are no missing values.

# Setup

Let's define the runtime options.

In [84]:
#usecase = 'scrape'
usecase = 'result exploration'

#usecase = 'update_results'
#usecase = 'update_search_configuration'
#usecase = 'hyperlink extraction'
#usecase = 'fix_links'
#usecase = 'check_links'

search_engine = 'pvp'
#search_engine = 'blinks'
#search_engine = 'guber'
#search_engine = 'blinks'

run_in_parallel = True
no_processes = 5


In [160]:
if search_engine == 'pvp':
    original_url_rootname = PVP_URL_ROOTNAME
    scraper = PVPScraper()
else:
    raise ValueError(f'Plese specify the correct value for {original_url_rootname}!')
print(original_url_rootname)

https://pvp.giustizia.it


In [15]:
rc_filename = os.path.expanduser('~/property_scraper.json')
with open(rc_filename, 'r') as f:
    configuration = json.load(f)
    print(configuration.keys())

dict_keys(['name', 'credentials', 'workbook_id', 'folder', 'JSON filename', 'search excel filename', 'search CSV filename', 'search parquet filename', 'search by location CSV filename', 'search by location parquet filename', 'search pattern', 'search property excel filename', 'search property CSV filename', 'search property parquet filename', 'search property pattern', 'property excel filename', 'property CSV filename', 'property parquet filename', 'property pattern', 'output folder', 'astagiudiziaria', 'astainsieme', 'astegiudiziarie', 'asteravenna', 'astetelematiche', 'avvisinotarili', 'canaleaste', 'idealista', 'iqera', 'immobiliare', 'pvp', 'quimmo', 'repubblica', 'subito', 'tribunale_laspezia', 'tribunale_savona', 'trovoaste', 'trovocasa'])


### Running a Scrapy Spider within Jupyter Notebook

To run a Scrapy spider within Jupyter Notebook, follow these steps:

1. **Install the necessary dependencies**: Make sure you have Scrapy and Jupyter Notebook installed in your Python environment. You can install them using pip:

In [7]:
#!python -m pip install scrapy jupyter

2. **Create a new Jupyter Notebook**: Launch Jupyter Notebook by running the command `jupyter notebook` in your terminal. Create a new notebook by clicking on "New" and selecting "Python 3" or use an existing notebook.

3. **Import the necessary modules**: In the first cell of your Jupyter Notebook, import the required modules.

4. **Define your Scrapy spider**: In a new cell, define your Scrapy spider as you normally would. Here's an example spider:

In [8]:
if usecase == 'scrape':
    spider = AsteGiudiziarieSearchSpider()

5. **Run the Scrapy spider**: In the next cell, run the Scrapy spider using the CrawlerProcess class and the project settings.

In [9]:
if usecase == 'scrape':
    #get_project_settings()
    #crawler = CrawlerProcess()#get_project_settings())
    crawler = CrawlerRunner()

In [10]:
if usecase == 'scrape':
    crawler.crawl(AsteGiudiziarieSearchSpider)

In [11]:
if usecase == 'scrape':
    pass
    #crawler.start()

6. **Execute the cells**: Run the cells in your Jupyter Notebook by clicking the "Run" button or using the keyboard shortcut. The Scrapy spider will start crawling the URLs specified in the **start_urls** attribute.

You should see the Scrapy spider's logs in the Jupyter Notebook output, indicating the progress of the crawling process. Once the spider has finished crawling, you can access and process the scraped data within the same notebook.

Note: Remember that Scrapy is primarily designed to be run from the command line, and running it within Jupyter Notebook may have some limitations and differences in behavior compared to running it in a traditional terminal environment.

Copy code
To run the code cells with Python code, you can use the Jupyter Notebook interface and execute the cells one by one by clicking the "Run" button or using the keyboard shortcut. This will start the Scrapy spider and display the crawling progress and any logs or errors in the output area.

After the spider has finished crawling, you can continue working in subsequent cells to process the scraped data or perform any additional tasks.

In [13]:
if usecase == 'scrape':
    !scrapy runspider /home/git/property_scraper/property_scraper/astegiudiziarie/spiders/search.py --output astegiudiziarie_search_default.csv

In [17]:
#print(json.dumps(configuration, indent=4))

# Update Search Configuration

In [19]:
if usecase == 'update_search_configuration':
    with open(json_filename, 'r') as f:
        data = json.load(f)

In [20]:
if usecase == 'update_search_configuration':
    url_rootname = data['root']
    print(url_rootname)

In [21]:
if usecase == 'update_search_configuration':
    rootname = ROOT_FOLDER
    print(rootname)

In [22]:
if usecase == 'update_search_configuration':
    data['searches'] = sorted(set(data['searches']))
    print(len(data['searches']))
    print('\n'.join(data['searches'][:10]))

In [23]:
if usecase == 'update_search_configuration':
    data['properties'] = sorted(set(data['properties']))
    print(len(data['properties']))
    print('\n'.join(data['properties'][:10]))

## Search

In [24]:
if usecase == 'update_search_configuration':
    filenames = glob(f'/home/git/property_scraper/www/{search_engine}/{search_engine}_search_*.html')
    filenames = sorted([os.path.relpath(x, '/home/git/property_scraper/www') for x in filenames])
    print(len(filenames))
    print('\n'.join(filenames[:10]))

In [25]:
if usecase == 'update_search_configuration':
    data['searches'] = sorted(set([x if x.startswith(f'{search_engine}/{search_engine}_search_') else f'{x[:2*len(search_engine) + 1]}_search{x[2*len(search_engine) + 1:]}' for x in filenames]))
    print(len(data['searches']))
    print('\n'.join(data['searches'][:10]))

## Property

In [26]:
if usecase == 'update_search_configuration':
    filenames = glob(f'/home/git/property_scraper/www/{search_engine}/{search_engine}_property_LTT*.html')
    filenames = sorted([os.path.relpath(x, '/home/git/property_scraper/www') for x in filenames])
    print(len(filenames))
    print('\n'.join(filenames[:10]))

In [27]:
if usecase == 'update_search_configuration':
    if 0 == 1:
        filenames_new = sorted(set([x if x.startswith(f'{search_engine}/{search_engine}_search_') else f'{x[:2*len(search_engine) + 1]}_property{x[2*len(search_engine) + 1:]}' for x in filenames]))
        print(len(filenames_new))
        filenames_new[:10]

In [28]:
if usecase == 'update_search_configuration':
    if 0 == 1:
        pbar = tqdm(range(len(filenames)))
        for ix in pbar:
            os.rename(os.path.join('/home/git/property_scraper/99_backup', filenames[ix]), 
                      os.path.join('/home/git/property_scraper/99_backup', filenames_new[ix]))

In [29]:
if usecase == 'update_search_configuration':
    data['properties'] = sorted(set([x if x.startswith(f'{search_engine}/{search_engine}_property_') else f'{x[:2*len(search_engine) + 1]}_search{x[2*len(search_engine) + 1:]}' for x in filenames]))
    print(len(data['properties']))
    print('\n'.join(data['properties'][:10]))

In [30]:
if usecase == 'update_search_configuration':
    with open(json_filename, 'w') as f:
        json.dump(data, f, indent=4)

In [31]:
if usecase == 'update_search_configuration':
    if 0 == 1:
        filename = f'/home/git/property_scraper/demos/{search_engine}_search_localhost.json'
        with open(filename, 'r') as f:
            data = json.load(f)
        #print(json.dumps(data, indent=4))

# Update Results

## Search

In [32]:
if usecase == 'update_results':
    store = SearchStore(name=rc['name'], credentials=rc['credentials'], workbook_id=rc['workbook_id'])

In [33]:
if usecase == 'update_results':
    store.results = store.update_results(rc['search parquet filename'], rc['search pattern'])

In [36]:
if usecase == 'update_results':
    store.fix_results()
    store.results.dtypes

In [38]:
if usecase == 'update_results':
    n = 5
    store.show(n)

Let's remove the incomplete searches.

In [39]:
if usecase == 'update_results':
    store.results_bylocation = store.create_results_bylocation_table()

In [40]:
if usecase == 'update_results':
    store.backup_results(rc['search parquet filename'], rc['search by location parquet filename'])

In [41]:
if usecase == 'update_results':
    store.backup_results(rc['search CSV filename'], rc['search by location CSV filename'])

In [42]:
if usecase == 'update_results':
    store.backup_results(rc['search excel filename'], '')

In [43]:
if usecase == 'update_results':
    store.to_gsheet(sheetname='search', bylocation_sheetname='search_bylocation')

Check whether the search page was downloaded.

In [44]:
if usecase == 'update_results':
    if 0 == 1:
        search['Scaricato?'] = search['Filename'].apply(lambda x: is_downloaded(x))
        display(search[~search['Scaricato?']].head(3).T)

Check whether the search page hyperlinks were fixed.

In [45]:
%%time

if usecase == 'update_results':
    if 0 == 1:
        pbar = tqdm(search[search['Scaricato?']].index)
        for ix in pbar:
            search.loc[ix, 'Hyperlink relativo riparato?'] = is_fixed(search.loc[ix, 'Filename'])
        print(search[~search['Hyperlink relativo riparato?']].shape)
        display(search[~search['Hyperlink relativo riparato?']].head(3).T)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10.5 µs


In [46]:
if usecase == 'update_results':
    if 0 == 1:
        print(search[~search['Hyperlink relativo riparato?']].shape)
        print(search.shape)

In [47]:
if usecase == 'update_results':
    if 0 == 1:
        basenames = [os.path.basename(x) for x in data['searches']]
        indices = search[search['Basename'].isin(basenames)].index
        search.loc[indices, 'Hyperlink relativo riparato?'] = True
        print(search[~search['Hyperlink relativo riparato?']].shape)
        display(search.head(3).T)

In [48]:
if usecase == 'update_results':
    if 0 == 1:
        df = search[search['Scaricato?'] & ~search['Hyperlink relativo riparato?']]
        display(df.head(3).T)

Let's enrich the search results.

In [49]:
if usecase == 'update_results':
    if 0 == 1:
        search['Data della Ricerca'] = search[['Basename']].apply(lambda x: get_datetime(x[0]), axis=1)
        #searches['data_della_ricerca'] = pd.to_datetime(searches['data_della_ricerca'])
        search['Data della Ricerca'] = search['Data della Ricerca'].map(lambda x: x.strftime('%d/%m/%Y'))

        url = search.iloc[0]['URL']
        #'https://pvp.giustizia.it/pvp/it/risultati_ricerca.page?localita=Palermo&geo=geografica&ordinamento=data_vendita_decre&view=list&frame4_item=63&categoria=&nazione=ITA&elementiPerPagina=50&ordine_localita=a_z&tipo_bene=immobili'
        parameters = get_parameters(url)
        print(parameters)

        for key in parameters.keys():
            search[key] = search[['URL']].apply(lambda x: get_parameter(key, x[0]), axis=1)

        search = search.reset_index()
        search = search.rename(columns=scraper.SEARCH_NAMES)
        search = search.set_index(['Identificativo'])

        #search["Localita' estratta dal nome del file"] = search["Localita' estratta dal nome del file"].astype(str)
        indices = search[pd.isnull(search["Localita' estratta dal nome del file"])].index
        search.loc[indices, "Localita' estratta dal nome del file"] = None

        print(search.shape)
        display(search.head(3).T)

Let's update the google sheet...

## Search Property

In [50]:
if usecase == 'update_results':
    store = SearchPropertyStore(name=rc['name'], credentials=rc['credentials'], workbook_id=rc['workbook_id'])

In [51]:
if usecase == 'update_results':
    store.results = store.update_results(rc['search property parquet filename'], rc['search property pattern'])

In [53]:
if usecase == 'update_results':
    store.fix_results()

    store.results.dtypes

In [55]:
if usecase == 'update_results':
    n = 5
    store.show(n)

In [56]:
if usecase == 'update_results':
    store.backup_results(rc['search property parquet filename'], None)

In [57]:
if usecase == 'update_results':
    store.backup_results(rc['search property CSV filename'], None)

In [58]:
if usecase == 'update_results':
    store.backup_results(rc['search excel filename'], None)

In [59]:
if usecase == 'update_results':
    store.to_gsheet(sheetname='search_property', bylocation_sheetname=None)

In [60]:
if usecase == 'update_results':
    if 0 == 1:
        for cid in ['indirizzo', 'numero_di_procedura']:
            search_property[cid] = search_property[cid].astype(str)
        for cid in ['is_downloaded']:
            search_property[cid] = search_property[cid].astype(bool)
        for cid in ['offerta_minima', 'prezzo_base']:
            search_property[cid] = search_property[cid].astype(float)
        for cid in ['response_status_code']:
            search_property[cid] = search_property[cid].astype(int)

## Property

In [61]:
if usecase == 'update_results':
    store = PropertyStore(name=rc['name'], credentials=rc['credentials'], workbook_id=rc['workbook_id'])

In [62]:
if usecase == 'update_results':
    store.results = store.update_results(rc['property parquet filename'], rc['property pattern'])

In [65]:
if usecase == 'update_results':
    store.fix_results()

    store.results.dtypes

In [66]:
if usecase == 'update_results':
    n = 5
    store.show(n)

In [67]:
if usecase == 'update_results':
    store.backup_results(rc['property parquet filename'], None)

In [68]:
if usecase == 'update_results':
    store.backup_results(rc['property CSV filename'], None)

In [69]:
if usecase == 'update_results':
    pass
    #store.backup_results(rc['property excel filename'], None)

In [70]:
if usecase == 'update_results':
    table_name = 'property'
    sql_statement = pd.io.sql.get_schema(store.results, table_name).replace('"', '`')#.replace('\n', '').replace('"', "")
    print(sql_statement)

In [77]:
if 0 == 1:
    filename = '/home/data/property_scraper/demos/pvp_property_localhost.enriched.sql'

    from sqlalchemy import create_engine
    engine = create_engine(f'sqlite://{filename}', echo=False)
    store.results.to_sql('property', con=engine, if_exists='append')

In [72]:
#stop here

Let's remove the incomplete searches.

In [73]:
if usecase == 'update_results':
    assert(len(sorted(store.results[store.results['indirizzo'] == 'False'].index)) == 0)

In [74]:
if usecase == 'update_results':
    assert(len(sorted(store.results[store.results['response_status_code'] != 200].index)) == 0)

In [75]:
if usecase == 'update_results':
    if 0 == 1:
        for cid in ['indirizzo']: #, 'numero_di_procedura']:
            property[cid] = property[cid].astype(str)
        for cid in ['is_downloaded']:
            property[cid] = property[cid].astype(bool)
        for cid in ['offerta_minima', 'prezzo_base']:
            property[cid] = property[cid].astype(float)
        for cid in ['response_status_code']:
            property[cid] = property[cid].astype(int)

In [76]:
#search.to_parquet(parquet_filename)

# Link Extractor

In [ ]:
if usecase == 'hyperlink extraction':
    dataframe = pd.DataFrame(worksheet.get_all_records())

## Search

In [78]:
if usecase == 'hyperlink extraction':
    worksheet = workbook.worksheet('search')

In [79]:
if usecase == 'hyperlink extraction':
    search = pd.DataFrame(worksheet.get_all_records())
    search = search.set_index('Identificativo')
    print(search.shape)
    display(search.head())

In [80]:
if usecase == 'hyperlink extraction':
    filenames = glob(os.path.join(rootname, 'pvp', 'pvp_search_*.html'))
    basenames = [os.path.basename(x) for x in filenames]
    print(f"{len(filenames)} -> \n\t" + '\n\t'.join(filenames[:10]))

In [81]:
if usecase == 'hyperlink extraction':
    search_missing = set(basenames) - set(search['Basename'].unique())
    print(len(search_missing))
    print('\n'.join(sorted(search_missing)[:5]))
    print('\n'.join(sorted(search_missing)[-5:]))

In [ ]:
if usecase == 'hyperlink extraction':
    if 0 == 1:
        repair_url('http://localhost/pvp/pvp_property_LTT4199300.html')

In [ ]:
if usecase == 'hyperlink extraction':
    urls = searches_to_be_downloaded(search_missing)
    print(f"{len(urls)} -> " + '\n'.join(sorted(urls)))

In [ ]:
if usecase == 'hyperlink extraction':
    if 0 == 1:
        pbar = tqdm(search.index)
        for idx in pbar:
            filename = search.loc[idx, 'Filename']
            links = get_links(filename)
            pbar.set_description(f"{filename:>80} -> {len(links):>10}")
            print('\n'.join(links[:10]))
            # http://localhost/pvp/pvp_property_LTT5906692.html -> 
            break

## Search Property

In [ ]:
if usecase == 'hyperlink extraction':
    worksheet = workbook.worksheet('search_property')

In [ ]:
if usecase == 'hyperlink extraction':
    search_property = pd.DataFrame(worksheet.get_all_records())
    search_property = search_property.set_index('Identificativo')
    print(search_property.shape)
    display(search_property.head())

In [ ]:
if usecase == 'hyperlink extraction':
    filenames = glob(os.path.join(rootname, 'pvp', 'pvp_property_*.html'))
    basenames = [os.path.basename(x) for x in filenames]
    print(f"{len(filenames)} -> \n\t" + '\n\t'.join(filenames[:10]))

In [ ]:
if usecase == 'hyperlink extraction':
    search_property_missing = set(basenames) - set(search_property['Basename'].unique())
    print(len(search_property_missing))
    print('\n'.join(sorted(search_property_missing)[:5]))
    print('\n'.join(sorted(search_property_missing)[-5:]))

In [ ]:
if usecase == 'hyperlink extraction':
    if 0 == 1:
        repair_url('http://localhost/pvp/pvp_property_LTT4199300.html')

In [ ]:
if usecase == 'hyperlink extraction':
    urls = searches_to_be_downloaded(search_property_missing)
    print(f"{len(urls)} -> " + '\n'.join(sorted(urls)))

## Property

In [ ]:
if usecase == 'hyperlink extraction':
    worksheet = workbook.worksheet('property')

In [ ]:
if usecase == 'hyperlink extraction':
    property = pd.DataFrame(worksheet.get_all_records())
    property['Basename'] = property['url'].apply(lambda x: os.path.basename(x))
    property['Identificativo'] = property['Basename'].apply(lambda x: os.path.splitext(x)[0].split('_')[-1].replace('LTT', ''))
    property = property.set_index('Identificativo')
    print(property.shape)
    display(property.head())

In [ ]:
if usecase == 'hyperlink extraction':
    filenames = glob(os.path.join(rootname, 'pvp', 'pvp_property_*.html'))
    basenames = [os.path.basename(x) for x in filenames]
    print(f"{len(filenames)} -> \n\t" + '\n\t'.join(filenames[:10]))

In [ ]:
if usecase == 'hyperlink extraction':
    property_missing = set(basenames) - set(property['Basename'].unique())
    print(len(property_missing))
    print('\n'.join(sorted(property_missing)[:5]))
    print('\n'.join(sorted(property_missing)[-5:]))

In [ ]:
if usecase == 'hyperlink extraction':
    if 0 == 1:
        repair_url('http://localhost/pvp/pvp_property_LTT4199300.html')

In [ ]:
if usecase == 'hyperlink extraction':
    urls = searches_to_be_downloaded(search_property_missing)
    print(f"{len(urls)} -> " + '\n'.join(sorted(urls)))

# Link Fixer

## Search

In [ ]:
if usecase == 'fix_links':
    parquet_filename = '/home/git/property_scraper/demos/pvp_search_default.enriched.parquet'
    search = pd.read_parquet(parquet_filename)
    print(search.shape)
    display(search.head(3).T)

In [ ]:
if usecase == 'fix_links':
    #_filenames = [x for x in sorted(data['searches'])]
    #print(f"{len(_filenames):10} -> {_filenames[:1]}")
    indices = search[~search['Scaricato?']].index
    filenames_to_be_downloaded = [os.path.join(search_engine, x) for x in sorted(search.loc[indices, 'Basename'].unique())]
    print(f"Number of Search Files to be downloaded: {len(filenames_to_be_downloaded):36} -> {filenames_to_be_downloaded[:1]}")
    indices = search[search['Scaricato?'] & search['Hyperlink relativo riparato?']].index
    filenames_fixed = [os.path.join(search_engine, x) for x in sorted(search.loc[indices, 'Basename'].unique())]
    print(f"Number of Search Files downloaded and with Hyperlinks fixed: {len(filenames_fixed):16} -> {filenames_fixed[:1]}")
    indices = search[search['Scaricato?'] & ~search['Hyperlink relativo riparato?']].index
    filenames_to_be_fixed = [os.path.join(search_engine, x) for x in sorted(search.loc[indices, 'Basename'].unique())]
    print(f"Number of Search Files downloaded but with Hyperlinks to be fixed: {len(filenames_to_be_fixed):10} -> {filenames_to_be_fixed[:1]}")
    if 0 == 1:
        #filenames = [x for x in sorted(data['searches']) if 'pvp/pvp_search_20230429' in x]
        #filenames = filenames[10:11]
        filenames_to_be_fixed = ['pvp/pvp_search_20230429072621_000069.html']
        print(filenames_to_be_fixed)

In [ ]:
def fix_search_hyperlinks(filename:str):
    prefixes = {
        '/': f'{PVP_URL_ROOTNAME}/',
        'prenotazione_visita.page?': f'{PVP_URL_ROOTNAME}/pvp/it/prenotazione_visita.page?',
        f'{PVP_URL_ROOTNAME}/pvp/en/': f'{PVP_URL_ROOTNAME}/pvp/it/',
        'http://localhost/pvp-resources/': f'{PVP_URL_ROOTNAME}/pvp-resources/',
        'http://localhost/pvp/it/': f'{PVP_URL_ROOTNAME}/pvp/it/'
    }
    
    func = {
        f'{PVP_URL_ROOTNAME}/pvp/it/dettaglio_annuncio.page?': get_filename_from_identifier,
        f'{PVP_URL_ROOTNAME}/pvp/en/dettaglio_annuncio.page?': get_filename_from_identifier
    }
    keys = ['contentId']
    root = f'http://localhost/{search_engine}/{search_engine}_property'
    
    path = os.path.join('/home/git/property_scraper/demos/downloads', filename)
    subfolder = f'../../../www/{search_engine}'
    text = None
    with open(path, 'r') as f:
        text = f.read()
        text = text.replace("'/pvp-resources/", f"'{PVP_URL_ROOTNAME}/pvp-resources/")
        text = text.replace('"/pvp-resources/', f'"{PVP_URL_ROOTNAME}/pvp-resources/')
        text = text.replace('http://localhost/pvp/de/', f'{PVP_URL_ROOTNAME}/pvp/de/')
        text = text.replace('http://localhost/pvp/en/', f'{PVP_URL_ROOTNAME}/pvp/en/')
        text = text.replace('http://localhost/pvp/it/', f'{PVP_URL_ROOTNAME}/pvp/it/')
        text = text.replace('http://localhost/pvp/en/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
        text = text.replace('http://localhost/pvp/it/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
        text = text.replace('http://localhost/pvp/it/dettaglio_annuncio.page?', f'{PVP_URL_ROOTNAME}/pvp/it/dettaglio_annuncio.page?')
        text = text.replace('http://localhost:8000/pvp/de/', f'{PVP_URL_ROOTNAME}/pvp/de/')
        text = text.replace('http://localhost:8000/pvp/en/', f'{PVP_URL_ROOTNAME}/pvp/en/')
        text = text.replace('http://localhost:8000/pvp/it/', f'{PVP_URL_ROOTNAME}/pvp/it/')
        text = text.replace('http://localhost:8000/pvp/en/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
        text = text.replace('http://localhost:8000/pvp/it/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
    if text:
        #print(f"File {path} was modified!")
        opath = os.path.realpath(os.path.join(os.path.dirname(path), subfolder, os.path.basename(path)))
        #print(f'Copying filename {path} to {opath} ... ')
        with open(opath, 'w') as of:
            of.write(text)
        fix_hyperlinks(opath, filters=func, keys=keys, root=root, url_rootname=url_rootname, prefixes=prefixes, overwrite=True)#, subfolder=subfolder)
        #break

In [ ]:
%%time

if usecase == 'fix_links':
    # Define the inputs to your function
    n0 = 0
    filenames = filenames_to_be_fixed[n0:]

    if run_in_parallel:
        # Create a multiprocessing pool
        with Pool(processes=no_processes) as mp_pool:
            # Use the pool to apply the function to each input in parallel
            results = []
            for result in tqdm(mp_pool.imap_unordered(fix_property_hyperlinks, filenames), total=len(filenames)):
                results.append(result)
    else:
        pbar = tqdm(filenames, position=0)
        #ix = 0
        for filename in pbar:
            pbar.set_description(f'{filename: >60}')
            fix_search_hyperlinks(filename)

## Property

In [ ]:
if usecase == 'fix_links':
    #filenames = [x for x in sorted(data['properties'])]
    filenames_to_be_fixed = glob('/home/git/property_scraper/demos/downloads/pvp/pvp_property_LTT*.html')
    filenames_to_be_fixed = [os.path.relpath(x, '/home/git/property_scraper/demos/downloads') for x in filenames_to_be_fixed]
    print(f"Number of Search Files downloaded but with Hyperlinks to be fixed: {len(filenames_to_be_fixed):10} -> {filenames_to_be_fixed[:1]}")    
    if 0 == 1:
        #filenames = [x for x in sorted(data['searches']) if 'pvp/pvp_search_20230429' in x]
        filenames = filenames[-10:]
        print(filenames)    

In [ ]:
def fix_property_hyperlinks(filename:str):
    prefixes = {
        '/': f'{PVP_URL_ROOTNAME}/',
        'prenotazione_visita.page?': f'{PVP_URL_ROOTNAME}/pvp/it/prenotazione_visita.page?',
        f'{PVP_URL_ROOTNAME}/pvp/en/': f'{PVP_URL_ROOTNAME}/pvp/it/',
        'http://localhost/pvp-resources/': f'{PVP_URL_ROOTNAME}/pvp-resources/',
        'http://localhost/pvp/it/': f'{PVP_URL_ROOTNAME}/pvp/it/',
        'http://localhost:8000/pvp/documents/': './documents/'
    }

    func = {
        f'{PVP_URL_ROOTNAME}/pvp-resources/cms/documents/': get_basename,

    }
    #https://pvp.giustizia.it/pvp-resources/cms/documents/2018/20181003/c54d1e6f718d6a7275bf317b0ebcc4b2.pdf
    keys = [None]
    root = f'http://localhost:8000/{search_engine}/documents/'
    
    path = os.path.join('/home/git/property_scraper/demos/downloads', filename)
    subfolder = f'../../../www/{search_engine}'
    text = None
    with open(path, 'r') as f:
        text = f.read()
        text = text.replace("'/pvp-resources/", f"'{PVP_URL_ROOTNAME}/pvp-resources/")
        text = text.replace('"/pvp-resources/', f'"{PVP_URL_ROOTNAME}/pvp-resources/')
        text = text.replace('http://localhost/pvp/de/', f'{PVP_URL_ROOTNAME}/pvp/de/')
        text = text.replace('http://localhost/pvp/en/', f'{PVP_URL_ROOTNAME}/pvp/en/')
        text = text.replace('http://localhost/pvp/it/', f'{PVP_URL_ROOTNAME}/pvp/it/')
        text = text.replace('http://localhost/pvp/en/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
        text = text.replace('http://localhost/pvp/it/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
        text = text.replace('http://localhost/pvp/it/dettaglio_annuncio.page?', f'{PVP_URL_ROOTNAME}/pvp/it/dettaglio_annuncio.page?')
        text = text.replace('http://localhost:8000/pvp/de/', f'{PVP_URL_ROOTNAME}/pvp/de/')
        text = text.replace('http://localhost:8000/pvp/en/', f'{PVP_URL_ROOTNAME}/pvp/en/')
        text = text.replace('http://localhost:8000/pvp/it/', f'{PVP_URL_ROOTNAME}/pvp/it/')
        text = text.replace('http://localhost:8000/pvp/en/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
        text = text.replace('http://localhost:8000/pvp/it/risultati_ricerca.page?', f'{PVP_URL_ROOTNAME}/pvp/it/risultati_ricerca.page?')
    if text:
        #print(f"File {path} was modified!")
        opath = os.path.realpath(os.path.join(os.path.dirname(path), subfolder, os.path.basename(path)))
        #print(f'Copying filename {path} to {opath} ... ')
        with open(opath, 'w') as of:
            of.write(text)
        fix_hyperlinks(opath, filters=func, keys=keys, root=root, url_rootname=url_rootname, prefixes=prefixes, overwrite=True) #, subfolder=subfolder')
    #break

In [ ]:
%%time

# Define the inputs to your function
n0 = 0 #231054
filenames = filenames_to_be_fixed[n0:]

if usecase == 'fix_links':
    if run_in_parallel:
        # Create a multiprocessing pool
        with Pool(processes=no_processes) as mp_pool:
            # Use the pool to apply the function to each input in parallel
            results = []
            for result in tqdm(mp_pool.imap_unordered(fix_property_hyperlinks, filenames), total=len(filenames)):
                results.append(result)
    else:
        pbar = tqdm(filenames_to_be_fixed[n0:], position=0)
        #ix = 0
        for filename in pbar:
            pbar.set_description(f'{filename: <50}')
            fix_property_hyperlinks(filename)

# Link Checker

## Search

In [ ]:
%%time 

if usecase == 'check_links':
    urls = data['searches']
    linkchecker(search_engine, urls, url_rootname, f'{search_engine}_search')

## Property

In [ ]:
%%time 

if usecase == 'check_links':
    urls = data['properties']
    linkchecker(search_engine, urls, url_rootname, f'{search_engine}_property')

In [ ]:
#!linkchecker --verbose --file-output=csv/../demos/linkchecker-test.csv http://localhost:8001/pvp/pvp_LTT1002377.html
#!linkchecker --check-extern --verbose --ignore-url=https://pvp.giustizia.it/pvp/it/ultimi_annunci.page --ignore-url=https://pvp.giustizia.it//analiticositi.giustizia.it/piwik.php?idsite=17 --output=csv http://localhost:8001/pvp/pvp_LTT1002377.html

In [ ]:
if usecase == 'check_links':
    filename = f'../demos/{search_engine}linkchecker-output.txt'

In [ ]:
if usecase == 'check_links':
    with open(filename, 'r') as f:
        for rowid, line in enumerate(f.readlines()):
            if rowid == 10:
                break
            print(line.strip())

In [ ]:
if usecase == 'check_links':
    links = pd.read_csv(filename, sep=';', comment='#', skiprows=[0, 1, 2, -1])
    display(links.head(3).T)

# Result Exploration

In [86]:
%%time 

if usecase == 'result exploration':
    print(configuration['property parquet filename'])
    properties = pd.read_parquet(configuration['property parquet filename'])
    display(properties.head())

/home/data/property_scraper/demos/pvp_property_localhost.enriched.parquet


,allegati,allegati_url,Basename,bene_descrizione,bene_disponibilita,bene_foglio,bene_particella,bene_piano,bene_sub,bene_tipologia,...,Codice dello Stato della Risposta,Rialzo minimo,Nome del Ragno,termine_presentazione_offerta,tipo_procedura,tipologia,tribunale,URL,Localhost URL,Batch
0,Relazione generale Mantova_Omissis.pdf (292 Kb...,/pvp-resources/cms/documents/2018/20181106/af7...,nan,LOTTO B9Area da urbanizzare di circa 3.286 mq ...,NaN,NaN,NaN,NaN,NaN,Senza incanto,...,200,0.0,pvp_property,13/12/2018 12:00,FALLIMENTARE (nuovo rito),Senza incanto,Tribunale di MANTOVA,http://localhost:8001/pvp/pvp_property_LTT1000...,http://localhost:8001/pvp/pvp_property_LTT1000...,
1,Avviso di vendita Spada 4 GEN 19 A.pdf (109 Kb...,https://pvp.giustizia.it/pvp-resources/cms/doc...,nan,Quota di proprietà del 50% di Appartamento al ...,NaN,NaN,NaN,NaN,NaN,Senza incanto,...,200,1000.0,pvp_property,03/01/2019 12:00,FALLIMENTARE (nuovo rito),Senza incanto,Tribunale di PESCARA,http://localhost:8001/pvp/pvp_property_LTT1000...,http://localhost:8001/pvp/pvp_property_LTT1000...,
2,"avviso di vendita.pdf (24 Kb),ordinanza.pdf (7...",https://pvp.giustizia.it/pvp-resources/cms/doc...,nan,Appartamento ad uso abitativo al piano terreno...,NaN,NaN,NaN,NaN,NaN,Senza incanto,...,200,1000.0,pvp_property,09/01/2019 12:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di BUSTO ARSIZIO,http://localhost:8001/pvp/pvp_property_LTT1000...,http://localhost:8001/pvp/pvp_property_LTT1000...,
3,"avviso12 161.07.pdf (235 Kb),ordinanza 161.07....",https://pvp.giustizia.it/pvp-resources/cms/doc...,nan,diritti di piena proprietà su porzione di fabb...,NaN,NaN,NaN,NaN,NaN,Senza incanto,...,200,0.0,pvp_property,04/12/2018 12:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di MACERATA,http://localhost:8001/pvp/pvp_property_LTT1000...,http://localhost:8001/pvp/pvp_property_LTT1000...,
4,"795821_perizia.pdf (3677 Kb),795820_ordinanza....",https://pvp.giustizia.it/pvp-resources/cms/doc...,nan,quota pari a 1/2 di appartamento al p.t. con l...,NaN,NaN,NaN,NaN,NaN,Senza incanto,...,200,1000.0,pvp_property,08/01/2019 12:00,FALLIMENTARE,Senza incanto,Tribunale di CATANZARO,http://localhost:8001/pvp/pvp_property_LTT1000...,http://localhost:8001/pvp/pvp_property_LTT1000...,


CPU times: user 2.27 s, sys: 1.32 s, total: 3.58 s
Wall time: 6.55 s


In [94]:
properties = properties.dropna(how='all').dropna(how='all', axis=1)

In [102]:
def to_list(s:str):
    if s:
        if ',' in s:
            return s.split(',')
        else:
            return [s]
    else:
        return [s]
    
for cid in ['allegati', 'allegati_url']:
    properties[cid] = properties[cid].apply(lambda x: to_list(x))

In [95]:
properties.shape

(247838, 38)

In [153]:
properties_next.head(27000).to_excel(configuration['property excel filename'])

In [128]:
datetime_str = '14/12/2018 11:00'

datetime_object = datetime.strptime(datetime_str, '%d/%m/%Y %H:%M')

print(type(datetime_object))
print(datetime_object)  # printed in default format

<class 'datetime.datetime'>
2018-12-14 11:00:00


In [145]:
for x in properties['Data di vendita'].unique():
    try:
        x_datetime = datetime.strptime(x, '%d/%m/%Y %H:%M')
    except:
        print(x, type(x))

In [155]:
#properties = properties.drop(indices)
#properties[pd.isnull(properties['Data di vendita'])]

,allegati,allegati_url,Basename,bene_descrizione,bene_tipologia,custode,custode_email,custode_telefono,custode_telefono_mobile,Data di vendita,...,Codice dello Stato della Risposta,Rialzo minimo,Nome del Ragno,termine_presentazione_offerta,tipo_procedura,tipologia,tribunale,URL,Localhost URL,Batch
3841,"[ordinanza 2018-.pdf (2030 Kb), quarto avviso....",[/pvp-resources/cms/documents/2018/20181218/6b...,nan,Mini appartamento al piano terra e primo.,Senza incanto,Riccardo Trento,None,None,None,NaT,...,200,2100.0,pvp_property,05/01/0001 00:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di FROSINONE,http://localhost:8001/pvp/pvp_property_LTT1224...,http://localhost:8001/pvp/pvp_property_LTT1224...,
11212,"[Avviso di vendita 3 luglio 2019.pdf (379 Kb),...",[/pvp-resources/cms/documents/2019/20190408/b7...,nan,"DIRITTI PARI ALLA PIENA PROPRIETÀ, IN RAGIONE ...",Senza incanto,Danilo Picano,danilo@danilopicano.it,077623479,3939032692,NaT,...,200,2000.0,pvp_property,02/07/2019 12:00,FALLIMENTARE (nuovo rito),Senza incanto,Tribunale di CASSINO,http://localhost:8001/pvp/pvp_property_LTT1794...,http://localhost:8001/pvp/pvp_property_LTT1794...,
38172,"[FOTO.pdf (234 Kb), CONVENZIONE.pdf (490 Kb), ...",[/pvp-resources/cms/documents/2020/20200622/d7...,nan,TERRENO EDIFICABILE RICADENTE ALL'INTERNO DEL ...,Competitiva,Michele Antonucci,antonucci.commercialista@gmail.com,0498762590,0498762590,NaT,...,200,2000.0,pvp_property,14/09/2020 12:00,FALLIMENTARE (nuovo rito),Competitiva,Tribunale di PADOVA,http://localhost:8001/pvp/pvp_property_LTT4078...,http://localhost:8001/pvp/pvp_property_LTT4078...,
63015,"[Ordinanza.pdf (796 Kb), Avviso Pubblicità 999...",[https://pvp.giustizia.it/pvp-resources/cms/do...,nan,"- al piano terreno (primo f.t.), con accesso d...",Senza incanto,Claudia D'Ambrosi,studiolegale@avvocatodambrosi.it,studiolegale@avvocatodambrosi.it,studiolegale@avvocatodambrosi.it,NaT,...,200,0.0,pvp_property,27/09/2018 12:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di TORINO,http://localhost:8001/pvp/pvp_property_LTT6057...,http://localhost:8001/pvp/pvp_property_LTT6057...,
219445,"[foto perizia_92_2022_ei.pdf (4204 Kb), delega...",[/pvp-resources/cms/documents/2023/20230324/02...,nan,unità immobiliare facente parte del fabbricato...,Senza incanto,Luca Lamanna,llamanna@notariato.it,0144325123,0144325123,NaT,...,200,500.0,pvp_property,04/07/2023 12:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di ALESSANDRIA,http://localhost:8001/pvp/pvp_property_LTT8016...,http://localhost:8001/pvp/pvp_property_LTT8016...,


In [157]:
properties['Data di vendita'] = pd.to_datetime(properties['Data di vendita'], errors = 'coerce')

indices = properties[pd.isnull(properties['Data di vendita'])].index
display(properties.loc[indices, :])

properties_next = properties[properties['Data di vendita'] >= datetime.today()]
print(properties_next.shape)

properties_past = properties[properties['Data di vendita'] < datetime.today()]
print(properties_past.shape)

,allegati,allegati_url,Basename,bene_descrizione,bene_tipologia,custode,custode_email,custode_telefono,custode_telefono_mobile,Data di vendita,...,Codice dello Stato della Risposta,Rialzo minimo,Nome del Ragno,termine_presentazione_offerta,tipo_procedura,tipologia,tribunale,URL,Localhost URL,Batch
3841,"[ordinanza 2018-.pdf (2030 Kb), quarto avviso....",[/pvp-resources/cms/documents/2018/20181218/6b...,nan,Mini appartamento al piano terra e primo.,Senza incanto,Riccardo Trento,None,None,None,NaT,...,200,2100.0,pvp_property,05/01/0001 00:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di FROSINONE,http://localhost:8001/pvp/pvp_property_LTT1224...,http://localhost:8001/pvp/pvp_property_LTT1224...,
11212,"[Avviso di vendita 3 luglio 2019.pdf (379 Kb),...",[/pvp-resources/cms/documents/2019/20190408/b7...,nan,"DIRITTI PARI ALLA PIENA PROPRIETÀ, IN RAGIONE ...",Senza incanto,Danilo Picano,danilo@danilopicano.it,077623479,3939032692,NaT,...,200,2000.0,pvp_property,02/07/2019 12:00,FALLIMENTARE (nuovo rito),Senza incanto,Tribunale di CASSINO,http://localhost:8001/pvp/pvp_property_LTT1794...,http://localhost:8001/pvp/pvp_property_LTT1794...,
38172,"[FOTO.pdf (234 Kb), CONVENZIONE.pdf (490 Kb), ...",[/pvp-resources/cms/documents/2020/20200622/d7...,nan,TERRENO EDIFICABILE RICADENTE ALL'INTERNO DEL ...,Competitiva,Michele Antonucci,antonucci.commercialista@gmail.com,0498762590,0498762590,NaT,...,200,2000.0,pvp_property,14/09/2020 12:00,FALLIMENTARE (nuovo rito),Competitiva,Tribunale di PADOVA,http://localhost:8001/pvp/pvp_property_LTT4078...,http://localhost:8001/pvp/pvp_property_LTT4078...,
63015,"[Ordinanza.pdf (796 Kb), Avviso Pubblicità 999...",[https://pvp.giustizia.it/pvp-resources/cms/do...,nan,"- al piano terreno (primo f.t.), con accesso d...",Senza incanto,Claudia D'Ambrosi,studiolegale@avvocatodambrosi.it,studiolegale@avvocatodambrosi.it,studiolegale@avvocatodambrosi.it,NaT,...,200,0.0,pvp_property,27/09/2018 12:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di TORINO,http://localhost:8001/pvp/pvp_property_LTT6057...,http://localhost:8001/pvp/pvp_property_LTT6057...,
219445,"[foto perizia_92_2022_ei.pdf (4204 Kb), delega...",[/pvp-resources/cms/documents/2023/20230324/02...,nan,unità immobiliare facente parte del fabbricato...,Senza incanto,Luca Lamanna,llamanna@notariato.it,0144325123,0144325123,NaT,...,200,500.0,pvp_property,04/07/2023 12:00,ESECUZIONE IMMOBILIARE POST LEGGE 80,Senza incanto,Tribunale di ALESSANDRIA,http://localhost:8001/pvp/pvp_property_LTT8016...,http://localhost:8001/pvp/pvp_property_LTT8016...,


(29335, 38)
(218058, 38)


In [103]:
properties = properties.rename(columns=scraper.PROPERTY_NAMES)
display(properties.head(3).T)

,0,1,2
allegati,[Relazione generale Mantova_Omissis.pdf (292 K...,[Avviso di vendita Spada 4 GEN 19 A.pdf (109 K...,"[avviso di vendita.pdf (24 Kb), ordinanza.pdf ..."
allegati_url,[/pvp-resources/cms/documents/2018/20181106/af...,[https://pvp.giustizia.it/pvp-resources/cms/do...,[https://pvp.giustizia.it/pvp-resources/cms/do...
Basename,nan,nan,nan
bene_descrizione,LOTTO B9Area da urbanizzare di circa 3.286 mq ...,Quota di proprietà del 50% di Appartamento al ...,Appartamento ad uso abitativo al piano terreno...
bene_tipologia,Senza incanto,Senza incanto,Senza incanto
custode,Sovemo Srl,Marco Giusti,Givg Srl
custode_email,immobiliaremantova@sovemo.com,marcogiusti@studiogiusti.com,visite.ba@givg.it
custode_telefono,0376220694,0856921244,0331322665
custode_telefono_mobile,0376220694,3316019558,0331322665
Data di vendita,14/12/2018 11:00,04/01/2019 09:00,10/01/2019 09:15
